In [1]:
!pip install -r requirements.txt

  Cloning https://github.com/TinArmEngineering/tinarm.git (to revision 0.1.57) to /private/var/folders/s5/09ndf2214qb_308ln1n6gfm40000gn/T/pip-req-build-4xik1i50
  Running command git clone --filter=blob:none --quiet https://github.com/TinArmEngineering/tinarm.git /private/var/folders/s5/09ndf2214qb_308ln1n6gfm40000gn/T/pip-req-build-4xik1i50
  Running command git checkout -q 65a15a5051ad593e6fd852426dbf5775ab5cc560
  Resolved https://github.com/TinArmEngineering/tinarm.git to commit 65a15a5051ad593e6fd852426dbf5775ab5cc560
  Preparing metadata (setup.py) ... done


In [2]:
import matplotlib.pyplot as plt
import pint
from tinarm import NameQuantityPair
from tinarm import Quantity 
from tinarm.api import JOB_STATUS

import tinarm
import logging
import time
import requests
import uuid
import random
import yaml
import pandas as pd
import pint_pandas


In [3]:

LOGGING_LEVEL = logging.INFO
STATUS_JOB = {value:key for key,value in JOB_STATUS.items()}

In [4]:

### Configure Logging
logger = logging.getLogger()
logger.setLevel(LOGGING_LEVEL)
logger.info(f"tinarm version {tinarm.__version__}")
logger.info(f"pint_pandas version {pint_pandas.__version__}")

q = pint_pandas.PintType.ureg
q.setup_matplotlib()

2024-01-29 19:38:29,739 - NoJobId - INFO - Martins-MacBook-Air.local - 1683099798.py-><module>() - tinarm version 0.1
2024-01-29 19:38:29,740 - NoJobId - INFO - Martins-MacBook-Air.local - 1683099798.py-><module>() - pint_pandas version 0.5


Log in to Tin Arm Engineering's machine solver, and under profile, retrieve your API key. 
Do not commit your API key to a repository, and consider it like a password.  A good way to keep it out of this code base is to use a configuation file, or environment variable. We will want other things cofigured too, so let's use a yaml file for convenience. 

create a `configurations.yaml` file in this directory with the content.
```yaml
api_key: 668952058c490d0a296da0abb966****
root_url: http://server-go:4300
```
But of course replace the number with your api key

You can then add that file to your `.gitignore` file by executing ```echo 'configurations.yaml' >> .gitignore```

```
!echo 'configurations.yaml' >> .gitignore
```

In [5]:
with open("configurations.yaml", "r") as f:
    config = yaml.safe_load(f)

# Stator

In [6]:
stator_parameters = {
    "slot_liner_thikness": 300 * q.um,
    "stator_bore": 8.20 * q.cm,
    "tooth_tip_depth": 1.5 * q.mm,
    "slot_opening": 1.5 * q.mm,
    "tooth_width": 9.8 * q.mm,
    "stator_outer_diameter": 0.136 * q.m,
    "back_iron_thickness": 5.5 * q.mm,
    "stator_internal_radius": 500 * q.um,
    "number_slots": 12 * q.count,
    "tooth_tip_angle": 70 * q.degrees
    }

air_gap_length = 1 * q.mm

In [7]:
from random import choices
from numpy.random import normal

n_variants = 30

sp_list = [stator_parameters.copy() for i in range(n_variants)]
keys = list(stator_parameters.keys())
z = zip(choices(population=keys, weights=[1 * k != "number_slots" for k in keys], k=n_variants), normal(loc=1.0, scale=0.4, size=n_variants))
for i, (k, v) in enumerate(z):
    print(k, v) 
    sp_list[i][k] = sp_list[i][k] * v

tooth_width 1.665837096549763
tooth_tip_angle 0.5940496922365157
slot_opening 1.012961100186465
slot_liner_thikness 0.8606089015392137
tooth_width 1.7954843563099245
slot_liner_thikness 0.5041453745552837
stator_internal_radius 0.9978290708927577
stator_bore 0.8827091225401007
tooth_tip_angle 0.7551008912898706
slot_liner_thikness 0.9819736714240667
slot_liner_thikness 0.802931289607328
stator_internal_radius 1.7008312325053243
stator_outer_diameter 0.9504808595478526
tooth_width 1.2548535072996716
stator_outer_diameter 0.4031878311731184
tooth_width 1.18082541972735
tooth_tip_angle 1.4991398515512433
stator_internal_radius -0.023832155959922385
back_iron_thickness 0.574946026952901
stator_outer_diameter 0.4149814551196441
slot_liner_thikness 0.03218016034263915
stator_outer_diameter 1.0895468783159354
slot_opening 1.5361365122499087
slot_liner_thikness 1.1993704985969074
stator_bore 1.18155174171775
slot_opening 1.1480187707051044
slot_opening 0.9284097691581006
back_iron_thickness 1.

In [8]:

sp_list

[{'slot_liner_thikness': 300 <Unit('micrometer')>,
  'stator_bore': 8.2 <Unit('centimeter')>,
  'tooth_tip_depth': 1.5 <Unit('millimeter')>,
  'slot_opening': 1.5 <Unit('millimeter')>,
  'tooth_width': 16.32520354618768 <Unit('millimeter')>,
  'stator_outer_diameter': 0.136 <Unit('meter')>,
  'back_iron_thickness': 5.5 <Unit('millimeter')>,
  'stator_internal_radius': 500 <Unit('micrometer')>,
  'number_slots': 12 <Unit('count')>,
  'tooth_tip_angle': 70 <Unit('degree')>},
 {'slot_liner_thikness': 300 <Unit('micrometer')>,
  'stator_bore': 8.2 <Unit('centimeter')>,
  'tooth_tip_depth': 1.5 <Unit('millimeter')>,
  'slot_opening': 1.5 <Unit('millimeter')>,
  'tooth_width': 9.8 <Unit('millimeter')>,
  'stator_outer_diameter': 0.136 <Unit('meter')>,
  'back_iron_thickness': 5.5 <Unit('millimeter')>,
  'stator_internal_radius': 500 <Unit('micrometer')>,
  'number_slots': 12 <Unit('count')>,
  'tooth_tip_angle': 41.5834784565561 <Unit('degree')>},
 {'slot_liner_thikness': 300 <Unit('micromet

## Rotor
Surface mounted Breadloaf magnets

In [9]:
rotor_parameters = {
    "rotor_od": stator_parameters["stator_bore"] - 2 * air_gap_length,
    "rotor_bore": 40 * q.mm,
    "banding_thickness": 0.5 * q.mm,
    "number_poles": 10 * q.count,
    "magnet_thickness": 4.5 * q.millimeter,
    "magnet_pole_arc": 150 * q.degrees,
    "magnet_inset": 0.25 * q.millimeter
    }

## Simulation Parameters

In [10]:
simulation_parameters = {
       "samples_per_electrical_period": 180 * q.count/q.turn,
        "timestep_intervals": 180 * q.count,
        "active_length": 65 * q.mm}


In [11]:
winding_parameters = {
    "symmetry": 2 * q.count,
    "number_phases": 3 * q.count,
    "number_layers": 2 * q.count,
    "coil_span": 1 * q.count,
    "turns_per_coil": 43 * q.count,
    "empty_slots": 0 * q.count,
    "fill_factor": 42 * q.percent
    }

## Operating Point

In [12]:
op_open_cct = {
    "simulated_speed": 2060 * q.rpm,
    "current_density": 0 * q.A / q.mm ** 2,
    "current_angle": 0 * q.degrees
    }

op_nominal = {
    'current_density': 6.23 * q.A * q.mm**-2,
    'current_angle': 255 * q.degrees,
    "simulated_speed": 2060 * q.rpm
    }

In [13]:
class Machine(object):
    def __init__(self, stator, rotor, winding):

        self.stator = stator
        self.rotor = rotor
        self.winding = winding

    def __repr__(self) -> str:
        return f"Machine({self.stator}, {self.rotor}, {self.winding})"
    
    def to_api(self):
        stator_api = [NameQuantityPair("stator",
                                        k,
                                        Quantity(*self.stator[k].to_tuple())) for k in  self.stator]
        rotor_api = [NameQuantityPair("rotor",
                                        k,
                                        Quantity(*self.rotor[k].to_tuple())) for k in  self.rotor]
        winding_api = [NameQuantityPair("winding",
                                        k,
                                        Quantity(*self.winding[k].to_tuple())) for k in  self.winding]
        data = []
        data.extend(list(x.to_dict() for x in stator_api))
        data.extend(list(x.to_dict() for x in rotor_api))
        data.extend(list(x.to_dict() for x in winding_api))
        return data
        
    
class Job(object):
    def __init__(self, machine: Machine, operating_point, simulation, title=None):
        if title is None:
            self.title = self.generate_title()
        else:
            self.title = title
        self.type = "electromagnetic_spmbrl_fscwseg"
        self.status = 0
        self.machine = machine
        self.operating_point = operating_point
        self.simulation = simulation
        
    def __repr__(self) -> str:
        return f"Job({self.machine}, {self.operating_point}, {self.simulation})"
    
    def generate_title(self):
        "gets a random title from the wordlists"
        random_offset = random.randint(500, 286797)
        response = requests.get("https://github.com/taikuukaits/SimpleWordlists/raw/master/Wordlist-Adjectives-All.txt",
                                headers={'Range': 'bytes={1}-{0}'.format(random_offset, random_offset-500), 'accept-encoding': 'identity'})
        adjective = random.choice(response.text.splitlines()[1:-1])
        random_offset = random.randint(500, 871742)
        response = requests.get("https://github.com/taikuukaits/SimpleWordlists/raw/master/Wordlist-Nouns-All.txt",
                                headers={'Range': 'bytes={1}-{0}'.format(random_offset, random_offset-500), 'accept-encoding': 'identity'})
        noun = random.choice(response.text.splitlines()[1:-1])
        title = f"{adjective}-{noun}"
        return title 

    def to_api(self):
        job =  {"status": 0,
                "title": self.title,
                "type": self.type,
                "tasks": 11,
                "data": []}
        
        operating_point_api = [NameQuantityPair("operating_point",
                                        k,
                                        Quantity(*self.operating_point[k].to_tuple())) for k in  self.operating_point]
        
        simulation_api = [NameQuantityPair("simulation",
                                        k,
                                        Quantity(*self.simulation[k].to_tuple())) for k in  self.simulation]
        job["data"].extend(list(x.to_dict() for x in operating_point_api))
        job["data"].extend(list(x.to_dict() for x in simulation_api))
        job["data"].extend(self.machine.to_api())
        return job
    
    def run(self):
        pass

In [14]:
class Api:
    """
    The TAE User API
    """

    def __init__(self, root_url, api_key, org_id=None):
        """
        Initialize the API
        """
        self._root_url = root_url
        self._api_key = api_key
        self._org_id = org_id
        self._node_id = None

        logger.info(f"root_url: {self._root_url}")

    def get_job(self, job_id):
        """
        Get a job from the TAE API
        """
        response = requests.get(
            url=f"{self._root_url}/jobs/{job_id}?apikey={self._api_key}",
        )
        response.raise_for_status()
        return response.json()
    
    def create_job(self, job):
        """
        Create a job for the TAE API
        """
        response = requests.post(
            url=f"{self._root_url}/jobs/?apikey={self._api_key}&org_id={self._org_id}",
            json=job.to_api()
        )
        response.raise_for_status()
        if response.status_code == 200:
            job.id = response.json()["id"]
        return response.json()
    
    def update_job_status(self, job_id, status):
        """
        Update a job status
        """
        url = f"{self._root_url}/jobs/{job_id}/status/{status}?node_id={self._node_id}&apikey={self._api_key}"
        logger.info(f"Updating job status: {url}")

        response = requests.put(url=url)
        return response.json()
    
    

In [15]:
api = Api(config["root_url"], config["api_key"], config["org_id"])

2024-01-29 19:38:29,915 - NoJobId - INFO - Martins-MacBook-Air.local - 3503141280.py->__init__() - root_url: https://api.build.tinarmengineering.com


In [16]:
machines = [Machine(sp, rotor_parameters, winding_parameters) for sp in sp_list]

jobs = [Job(m, op_open_cct, simulation_parameters) for m in machines]

jobs_mapping = map(lambda x: api.create_job(x), jobs)
jobs_result = list(jobs_mapping)



In [17]:
for j in jobs_result:
    res = api.update_job_status(j['id'], JOB_STATUS['QueuedForMeshing'])
    time.sleep(0.5)
    print(res)


2024-01-29 19:39:00,039 - NoJobId - INFO - Martins-MacBook-Air.local - 3503141280.py->update_job_status() - Updating job status: https://api.build.tinarmengineering.com/jobs/65b7f0b7ed812aadd4af3e99/status/10?node_id=None&apikey=668952058c490d0a296da0abb9660534
{'id': '65b7f0b7ed812aadd4af3e99', 'owner_org_id': '65a9291658b95253163001ab', 'owner_user_id': '659bf5a7b7135efed48311bd', 'creation_date': '2024-01-29T18:38:47Z', 'status': 10, 'title': 'armipotent-pivoting', 'type': 'electromagnetic_spmbrl_fscwseg', 'node_id': 'None', 'data': [{'section': 'operating_point', 'name': 'simulated_speed', 'value': {'magnitude': 2060, 'units': [{'name': 'revolutions_per_minute', 'exponent': 1}]}}, {'section': 'operating_point', 'name': 'current_density', 'value': {'magnitude': 0, 'units': [{'name': 'ampere', 'exponent': 1}, {'name': 'millimeter', 'exponent': -2}]}}, {'section': 'operating_point', 'name': 'current_angle', 'value': {'magnitude': 0, 'units': [{'name': 'degree', 'exponent': 1}]}}, {'se

In [18]:

for j in jobs_result:
    if STATUS_JOB[api.get_job(j['id'])['status']] == "Quarantined"
    

SyntaxError: expected ':' (3275868147.py, line 2)